In [ ]:
from crossword_solver.crossword_detection import detect_crossword_from_file
from copy import deepcopy
from crossword_solver.solver_helpers import find_suitable_candidates, find_whole_crossword_candidates, reorder_crossword_hints
import time
from collections import defaultdict
import numpy as np

In [ ]:
path = '../data/pictures/ristsona_1.png'
crossword = detect_crossword_from_file(path)

In [ ]:
find_whole_crossword_candidates(crossword)
reorder_crossword_hints(crossword)

In [ ]:
def solving_algorithm(crossword, max_empty_words = 2):
    if len(crossword.hints)==0:
        yield np.copy(crossword.matrix), crossword.score
  
    hint = crossword.hints.pop(0)
    suitable_candidates = find_suitable_candidates(hint, crossword)

    if len(suitable_candidates)==0 and max_empty_words==0:
        yield np.copy(crossword.matrix), crossword.score
    if max_empty_words>0:
        yield from solving_algorithm(crossword, max_empty_words-1)
    for candidate in suitable_candidates:
        prev_text = np.copy(crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1])
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = np.array(list(candidate.text)
                                                                                          ).reshape(hint.x_max+1-hint.x_min, -1)
        crossword.score += candidate.weight
        yield from solving_algorithm(crossword, max_empty_words)
        crossword.score -= candidate.weight
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = prev_text
    crossword.hints.insert(0, hint)

In [ ]:
results = list(solving_algorithm(crossword, max_empty_words=5))
print(len(results))

In [ ]:
printing_crossword = deepcopy(crossword)
for matrix, score in sorted(results, key=lambda x: x[1], reverse = True)[:5]:
    printing_crossword.matrix = matrix
    print(printing_crossword, score)